In [165]:
# import os
# os.chdir("birthrate_mtgp")
from jax import numpy as jnp
import numpy as np
import numpyro.distributions as dist
import jax.numpy as jnp
import numpyro
from numpyro.handlers import scope

## Age

In [166]:
import pandas as pd

# File paths
path_fertility = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data with 2024.csv"
path_2024 = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Mother Age.csv"

# Step 1: Load both datasets
df_fert = pd.read_csv(path_fertility)
df_2024 = pd.read_csv(path_2024)

# Step 2: Standardize column names
df_2024.columns = df_2024.columns.str.strip().str.lower()
df_2024.rename(columns={'month': 'month', 'state': 'state', 'motherage': 'age', 'births': 'births'}, inplace=True)

# Step 3: Map age groups to match target columns
def map_age_group(age):
    if age in ['15-19', '20-24']:
        return 'births_age1524'
    elif age in ['25-29', '30-34']:
        return 'births_age2534'
    elif age in ['35-39', '40-44']:
        return 'births_age3544'
    else:
        return None

df_2024['age_group'] = df_2024['age'].apply(map_age_group)
df_2024 = df_2024.dropna(subset=['age_group'])

# Step 4: Compute bimonthly code (bmcode: 1 for Jan-Feb, ..., 6 for Nov-Dec)
df_2024['bmcode'] = ((df_2024['month'] - 1) // 2 + 1).astype(int)

# Step 5: Aggregate to state-bmcode-age_group level
df_agg = df_2024.groupby(['state', 'bmcode', 'age_group'])['births'].sum().unstack('age_group').reset_index()
df_agg['year'] = 2023

# Step 6: Reorder and match column order with original dataset
df_agg = df_agg[['state', 'year', 'bmcode', 'births_age1524', 'births_age2534', 'births_age3544']]

# Step 7: Replace 2023 rows in original data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_updated = pd.concat([df_fert_no2023, df_agg], ignore_index=True)

# Step 8: Save updated dataset
output_path = path_fertility.replace("fertility_data with 2024.csv", "fertility_data_updated.csv")
df_updated.to_csv(output_path, index=False)
print("✅ Updated data saved to:", output_path)


✅ Updated data saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated.csv


## Marital Status

In [167]:
import pandas as pd

# File paths
fertility_updated_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated.csv"
marital_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Marital Status.csv"

# Step 1: Load datasets
df_fert = pd.read_csv(fertility_updated_path)
df_marital = pd.read_csv(marital_path)

# Step 2: Standardize column names
df_marital.columns = df_marital.columns.str.strip().str.lower()

# Step 3: Map marital status to correct column name
df_marital['marital_col'] = df_marital['marital_status'].map({
    'Married': 'births_married',
    'Unmarried': 'births_unmarried'
})

# Drop unknown statuses
df_marital = df_marital.dropna(subset=['marital_col'])

# Step 4: Calculate bmcode (1 to 6)
df_marital['bmcode'] = ((df_marital['month'] - 1) // 2 + 1).astype(int)

# Step 5: Pivot to wide format: one row per (state, bmcode), columns: births_married, births_unmarried
df_marital_pivot = (
    df_marital
    .groupby(['state', 'bmcode', 'marital_col'])['births']
    .sum()
    .unstack('marital_col')
    .reset_index()
)

# Add year column
df_marital_pivot['year'] = 2023

# Step 6: Keep relevant columns in order
df_marital_pivot = df_marital_pivot[['state', 'year', 'bmcode', 'births_married', 'births_unmarried']]

# Step 7: Merge with previous data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]

# First merge previous age-group replacement
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['births_married', 'births_unmarried'], errors='ignore')

# Now merge with marital data
df_merged = pd.merge(
    df_fert_2023,
    df_marital_pivot,
    on=['state', 'year', 'bmcode'],
    how='left'
)

# Step 8: Combine everything
df_final = pd.concat([df_fert_no2023, df_merged], ignore_index=True)

# Step 9: Save final dataset
output_path = fertility_updated_path.replace("fertility_data_updated.csv", "fertility_data_updated_v2.csv")
df_final.to_csv(output_path, index=False)
print("✅ Final updated file saved to:", output_path)


✅ Final updated file saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v2.csv


## Insurance

In [168]:
import pandas as pd

# File paths
fertility_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v2.csv"
insurance_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Insurance.csv"

# Step 1: Load data
df_fert = pd.read_csv(fertility_path)
df_ins = pd.read_csv(insurance_path)

# Step 2: Standardize column names
df_ins.columns = df_ins.columns.str.strip().str.lower()
if 'insurance' not in df_ins.columns:
    print("❌ Error: Could not find 'insurance' column. Found:", df_ins.columns.tolist())
    raise

# Step 3: Classify insurance types
df_ins['insurance_group'] = df_ins['insurance'].apply(
    lambda x: 'births_medicaid' if x.strip().lower() == 'medicaid' else 'births_nonmedicaid'
)

# Step 4: Compute bmcode (Jan-Feb = 1, Mar-Apr = 2, ..., Nov-Dec = 6)
df_ins['bmcode'] = ((df_ins['month'] - 1) // 2 + 1).astype(int)

# Step 5: Aggregate by state-bmcode-insurance_group
df_ins_agg = (
    df_ins
    .groupby(['state', 'bmcode', 'insurance_group'])['births']
    .sum()
    .unstack('insurance_group')
    .reset_index()
)

df_ins_agg['year'] = 2023

# Step 6: Reorder columns
df_ins_agg = df_ins_agg[['state', 'year', 'bmcode', 'births_medicaid', 'births_nonmedicaid']]

# Step 7: Separate and merge with existing 2023 data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['births_medicaid', 'births_nonmedicaid'], errors='ignore')

# Merge on keys
df_merged = pd.merge(df_fert_2023, df_ins_agg, on=['state', 'year', 'bmcode'], how='left')

# Step 8: Combine all rows and save
df_final = pd.concat([df_fert_no2023, df_merged], ignore_index=True)

output_path = fertility_path.replace("fertility_data_updated_v2.csv", "fertility_data_updated_v3.csv")
df_final.to_csv(output_path, index=False)
print("✅ Insurance update complete. Saved to:", output_path)


✅ Insurance update complete. Saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v3.csv


## Total

In [169]:
import pandas as pd

# File paths
fertility_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v3.csv"
total_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Total Births.csv"

# Step 1: Load data
df_fert = pd.read_csv(fertility_path)
df_total = pd.read_csv(total_path)

# Step 2: Standardize column names
df_total.columns = df_total.columns.str.strip().str.lower()

# Step 3: Compute bimonthly code
df_total['bmcode'] = ((df_total['month'] - 1) // 2 + 1).astype(int)

# Step 4: Aggregate births by state and bmcode
df_total_agg = df_total.groupby(['state', 'bmcode'])['births'].sum().reset_index()
df_total_agg['year'] = 2023
df_total_agg.rename(columns={'births': 'births_total'}, inplace=True)

# Step 5: Prepare the 2023 portion of fertility data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['births_total'], errors='ignore')

# Step 6: Merge the new total births into 2023 portion
df_fert_2023_updated = pd.merge(df_fert_2023, df_total_agg, on=['state', 'year', 'bmcode'], how='left')

# Step 7: Combine and save
df_final = pd.concat([df_fert_no2023, df_fert_2023_updated], ignore_index=True)

output_path = fertility_path.replace("fertility_data_updated_v3.csv", "fertility_data_updated_v4.csv")
df_final.to_csv(output_path, index=False)
print("✅ Total births updated and saved to:", output_path)


✅ Total births updated and saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v4.csv


## Age Group

In [170]:
import pandas as pd

# File paths
fertility_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v4.csv"
pop_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Population by Age.csv"

# Step 1: Load datasets
df_fert = pd.read_csv(fertility_path)
df_pop = pd.read_csv(pop_path)

# Step 2: Standardize column names
df_pop.columns = df_pop.columns.str.strip().str.lower()
df_pop.rename(columns={'sate': 'state', 'age group': 'age_group'}, inplace=True)

# Step 3: Map age groups to fertility categories
def map_age_group(age):
    if age in ['15-19', '20-24']:
        return 'pop_age1524'
    elif age in ['25-29', '30-34']:
        return 'pop_age2534'
    elif age in ['35-39', '40-44']:
        return 'pop_age3544'
    else:
        return None

df_pop['pop_group'] = df_pop['age_group'].apply(map_age_group)
df_pop = df_pop.dropna(subset=['pop_group'])

# Step 4: Aggregate to state + pop_group
df_pop_agg = (
    df_pop.groupby(['state', 'pop_group'])['population']
    .sum()
    .unstack('pop_group')
    .reset_index()
)
df_pop_agg['year'] = 2023  # Population for all bmcode in 2023

# Step 5: Apply this to every bmcode row in 2023 fertility data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['pop_age1524', 'pop_age2534', 'pop_age3544'], errors='ignore')

# Merge: many-to-one on state and year
df_fert_2023_updated = pd.merge(df_fert_2023, df_pop_agg, on=['state', 'year'], how='left')

# Step 6: Combine and save
df_final = pd.concat([df_fert_no2023, df_fert_2023_updated], ignore_index=True)

output_path = fertility_path.replace("fertility_data_updated_v4.csv", "fertility_data_updated_v5.csv")
df_final.to_csv(output_path, index=False)
print("✅ Population updated and saved to:", output_path)


✅ Population updated and saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v5.csv


## Total

In [171]:
import pandas as pd

# File paths
fertility_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v5.csv"
pop_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Population by Age.csv"

# Step 1: Load datasets
df_fert = pd.read_csv(fertility_path)
df_pop = pd.read_csv(pop_path)

# Step 2: Standardize column names
df_pop.columns = df_pop.columns.str.strip().str.lower()
df_pop.rename(columns={'sate': 'state', 'age group': 'age_group'}, inplace=True)

# Step 3: Sum all population by state (regardless of age group)
df_total_pop = df_pop.groupby('state')['population'].sum().reset_index()
df_total_pop['year'] = 2023
df_total_pop.rename(columns={'population': 'pop_total'}, inplace=True)

# Step 4: Prepare 2023 data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_fert_2023 = df_fert[df_fert['year'] == 2023].drop(columns=['pop_total'], errors='ignore')

# Step 5: Merge total population to 2023 fertility rows
df_fert_2023_updated = pd.merge(df_fert_2023, df_total_pop, on=['state', 'year'], how='left')

# Step 6: Concatenate and save
df_final = pd.concat([df_fert_no2023, df_fert_2023_updated], ignore_index=True)

output_path = fertility_path.replace("fertility_data_updated_v5.csv", "fertility_data_updated_v6.csv")
df_final.to_csv(output_path, index=False)
print("✅ pop_total updated and saved to:", output_path)


✅ pop_total updated and saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v6.csv


In [172]:
import pandas as pd

# File paths (adjust if needed)
original_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data with 2024.csv"
updated_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated_v6.csv"
output_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_birth_update.csv"

# Load datasets
df_original = pd.read_csv(original_path)
df_updated = pd.read_csv(updated_path)

# Only update these columns (exclude population columns)
columns_to_update = [
    'births_age1524', 'births_age2534', 'births_age3544',
    'births_married', 'births_unmarried',
    'births_medicaid', 'births_nonmedicaid',
    'births_total'
]

# Filter to 2023 rows
df_original_2023 = df_original[df_original['year'] == 2023]
df_non_2023 = df_original[df_original['year'] != 2023]
df_updated_2023 = df_updated[df_updated['year'] == 2023][['state', 'year', 'bmcode'] + columns_to_update]

# Merge updates into 2023 data
df_merged_2023 = pd.merge(df_original_2023, df_updated_2023, on=['state', 'year', 'bmcode'], how='left', suffixes=('', '_new'))

# Replace old values with updated ones
for col in columns_to_update:
    new_col = f"{col}_new"
    if new_col in df_merged_2023.columns:
        df_merged_2023[col] = df_merged_2023[new_col]
        df_merged_2023.drop(columns=[new_col], inplace=True)

# Combine and save
df_final = pd.concat([df_non_2023, df_merged_2023], ignore_index=True)
df_final.to_csv(output_path, index=False)

print(f"✅ Saved updated file as: {output_path}")


✅ Saved updated file as: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_birth_update.csv


In [152]:
import pandas as pd

# File paths
original_path = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data with 2024.csv"
updated_path = original_path.replace("fertility_data with 2024.csv", "fertility_data_updated_v6.csv")
final_output_path = original_path.replace("fertility_data with 2024.csv", "fertility_data_fully_updated.csv")

# Load original and updated datasets
df_original = pd.read_csv(original_path)
df_updated = pd.read_csv(updated_path)

# Separate out the 2023 data in the original file
df_original_no2023 = df_original[df_original['year'] != 2023]

# Combine with the updated 2023 data
df_updated_2023 = df_updated[df_updated['year'] == 2023]

# Concatenate and save final output
df_final = pd.concat([df_original_no2023, df_updated_2023], ignore_index=True)
df_final.to_csv(final_output_path, index=False)

import ace_tools as tools; tools.display_dataframe_to_user(name="Fully Updated Fertility Data", dataframe=df_final)


ModuleNotFoundError: No module named 'ace_tools'

## Race

In [173]:
import pandas as pd

files = [
    "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/race 2024/2024 births_hisp.csv",
    "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/race 2024/2024 births_nhblack.csv",
    "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/race 2024/2024 births_nhwhite.csv"
]

df0 = pd.concat([pd.read_csv(f) for f in files]).pivot(index=['state','month'], columns='race', values='births').reset_index()
df0.to_csv("/Users/shaokangyang/Downloads/2024_births_merged.csv", index=False)
df = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_birth_update.csv")


In [174]:
import pandas as pd

# Ensure df0 race columns match df's names
df0 = df0.rename(columns={'hisp':'births_hisp', 'nhblack':'births_nhblack', 'nhwhite':'births_nhwhite'})
cols = ['births_hisp','births_nhblack','births_nhwhite']

# Collapse df0 to bmcode (1=Jan–Feb, …, 6=Nov–Dec)
bim = (df0.assign(bmcode=((df0['month']-1)//2)+1)
          .groupby(['state','bmcode'], as_index=False)[cols].sum())

# Overwrite 2023 values in df using the bi-monthly sums
m = df['year'].eq(2023)
df.loc[m, cols] = (
    df.loc[m, ['state','bmcode']]
      .merge(bim, on=['state','bmcode'], how='left')[cols]
      .to_numpy()
)

m = df['year'].eq(2023)
df.loc[m, 'births_otherraceeth'] = df.loc[m, 'births_total'] - df.loc[m, ['births_hisp','births_nhblack','births_nhwhite']].sum(axis=1)



In [ ]:
df2023 = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data.csv")

In [43]:
df2023[['state', 'year', 'bmcode', "births_hisp","births_nhblack","births_nhwhite","births_otherraceeth", "births_total"]]

,state,year,bmcode,births_hisp,births_nhblack,births_nhwhite,births_otherraceeth,births_total
0,Alabama,2016,1,729.0,2837.0,5627.0,316.0,9509.0
1,Alabama,2016,2,733.0,2727.0,5562.0,312.0,9334.0
2,Alabama,2016,3,725.0,2841.0,5978.0,288.0,9832.0
3,Alabama,2016,4,805.0,3071.0,6113.0,364.0,10353.0
4,Alabama,2016,5,795.0,3043.0,6005.0,360.0,10203.0
...,...,...,...,...,...,...,...,...
2494,Wyoming,2023,3,150.0,NaN,812.0,NaN,1051.0
2495,Wyoming,2023,4,143.0,NaN,817.0,NaN,1065.0
2496,Wyoming,2023,5,153.0,NaN,743.0,NaN,986.0
2497,Wyoming,2023,6,126.0,10.0,714.0,90.0,940.0


In [84]:
df[['state', 'year', 'bmcode', "births_hisp","births_nhblack","births_nhwhite","births_otherraceeth", "births_total"]]

,state,year,bmcode,births_hisp,births_nhblack,births_nhwhite,births_otherraceeth,births_total
0,Alabama,2016,1,729.0,2837.0,5627.0,316.0,9509.0
1,Alabama,2016,2,733.0,2727.0,5562.0,312.0,9334.0
2,Alabama,2016,3,725.0,2841.0,5978.0,288.0,9832.0
3,Alabama,2016,4,805.0,3071.0,6113.0,364.0,10353.0
4,Alabama,2016,5,795.0,3043.0,6005.0,360.0,10203.0
...,...,...,...,...,...,...,...,...
2494,Wyoming,2023,2,158.0,0.0,622.0,270.0,1050.0
2495,Wyoming,2023,3,150.0,0.0,655.0,245.0,1050.0
2496,Wyoming,2023,4,168.0,0.0,640.0,263.0,1071.0
2497,Wyoming,2023,5,151.0,0.0,600.0,256.0,1007.0


## Education

In [175]:
import pandas as pd, glob

files = glob.glob("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/edu 2024/2024 births_*.csv")
df0 = pd.concat([pd.read_csv(f) for f in files]).pivot(index=['state','month'], columns='education', values='births').reset_index()
df0.to_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/edu_2024.csv", index=False)


In [176]:
import pandas as pd

# Ensure df0 column names match df
df0 = df0.rename(columns={'coll':'births_coll','hs':'births_hs','nohs':'births_nohs','somecoll':'births_somecoll'})
cols = ['births_coll','births_hs','births_nohs','births_somecoll']

# Collapse df0 to bmcode (1=Jan–Feb,...,6=Nov–Dec)
bim = (df0.assign(bmcode=((df0['month']-1)//2)+1)
          .groupby(['state','bmcode'], as_index=False)[cols].sum())

# Overwrite only 2023 rows in df by matching on state, bmcode
m = df['year'].eq(2023)
df.loc[m, cols] = (df.loc[m, ['state','bmcode']]
                     .merge(bim, on=['state','bmcode'], how='left')[cols]
                     .to_numpy())

# (optional) save
df.to_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_birth_update.csv", index=False)


In [88]:
df2023[['state', 'year', 'bmcode', 'births_coll','births_hs','births_nohs','births_somecoll', "births_total"]]

,state,year,bmcode,births_coll,births_hs,births_nohs,births_somecoll,births_total
0,Alabama,2016,1,2152.0,2938.0,1549.0,2851.0,9509.0
1,Alabama,2016,2,2298.0,2845.0,1425.0,2751.0,9334.0
2,Alabama,2016,3,2491.0,2877.0,1500.0,2941.0,9832.0
3,Alabama,2016,4,2419.0,3224.0,1594.0,3097.0,10353.0
4,Alabama,2016,5,2402.0,3147.0,1611.0,3030.0,10203.0
...,...,...,...,...,...,...,...,...
2494,Wyoming,2023,3,323.0,247.0,85.0,379.0,1051.0
2495,Wyoming,2023,4,310.0,271.0,86.0,372.0,1065.0
2496,Wyoming,2023,5,282.0,254.0,83.0,352.0,986.0
2497,Wyoming,2023,6,254.0,226.0,77.0,353.0,940.0


In [91]:
df[['state', 'year', 'bmcode', 'births_coll','births_hs','births_nohs','births_somecoll', "births_total"]]

,state,year,bmcode,births_coll,births_hs,births_nohs,births_somecoll,births_total
0,Alabama,2016,1,2152.0,2938.0,1549.0,2851.0,9509.0
1,Alabama,2016,2,2298.0,2845.0,1425.0,2751.0,9334.0
2,Alabama,2016,3,2491.0,2877.0,1500.0,2941.0,9832.0
3,Alabama,2016,4,2419.0,3224.0,1594.0,3097.0,10353.0
4,Alabama,2016,5,2402.0,3147.0,1611.0,3030.0,10203.0
...,...,...,...,...,...,...,...,...
2494,Wyoming,2023,2,315.0,251.0,101.0,338.0,1050.0
2495,Wyoming,2023,3,325.0,259.0,88.0,352.0,1050.0
2496,Wyoming,2023,4,320.0,282.0,89.0,337.0,1071.0
2497,Wyoming,2023,5,306.0,270.0,88.0,350.0,1007.0


## Exposure Update

In [188]:
df1 = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data.csv")
df2 = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_birth_update.csv")

In [178]:
import pandas as pd

# Example: ensure df2's policy_on, exposed_births, dobbs_code match df1
cols_to_update = ["policy_on", "exposed_births", "dobbs_code"]

df2 = df2.drop(columns=cols_to_update, errors="ignore").merge(
    df1[["state", "year", "bmcode"] + cols_to_update],
    on=["state", "year", "bmcode"],
    how="left"
)


In [179]:
# total births limited to age 15-44
df2['births_total'] = df2['births_age1524'] + df2['births_age2534'] + df2['births_age3544']

In [180]:
df2.to_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_birth_update.csv",index=False)

In [181]:
import pandas as pd

# Read files
df1 = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data.csv")
df2 = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_birth_update.csv")

# Find rows in df1 not in df2 based on key columns
diff_rows = df1.merge(df2[['state', 'year', 'bmcode']], 
                      on=['state', 'year', 'bmcode'], 
                      how='left', 
                      indicator=True).query('_merge == "left_only"').drop(columns='_merge')

print(diff_rows)


Empty DataFrame
Columns: [state, year, bmcode, births_age1524, births_age2534, births_age3544, births_nohs, births_hs, births_somecoll, births_coll, births_married, births_unmarried, births_medicaid, births_nhblack, births_nhwhite, births_hisp, births_total, births_otherraceeth, births_nonmedicaid, pop_total, pop_nhblack, pop_nhwhite, pop_hisp, pop_otherraceeth, pop_medicaid, pop_nonmedicaid, pop_nohs, pop_hs, pop_somecoll, pop_coll, pop_married, pop_unmarried, pop_age1524, pop_age2534, pop_age3544, policy_on, exposed_births, start_date, end_date, time, dobbs_code]
Index: []


In [185]:
df1 = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/fertility_results/df_through_june.csv")
df2 = pd.read_csv("/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/fertility_results/2024/df_through_june.csv")

In [189]:
pd.set_option('display.max_columns', None)
df1.describe()

,year,bmcode,births_age1524,births_age2534,births_age3544,births_nohs,births_hs,births_somecoll,births_coll,births_married,births_unmarried,births_medicaid,births_nhblack,births_nhwhite,births_hisp,births_total,births_otherraceeth,births_nonmedicaid,pop_total,pop_nhblack,pop_nhwhite,pop_hisp,pop_otherraceeth,pop_medicaid,pop_nonmedicaid,pop_nohs,pop_hs,pop_somecoll,pop_coll,pop_married,pop_unmarried,pop_age1524,pop_age2534,pop_age3544,policy_on,exposed_births,dobbs_code
count,2499.000000,2499.000000,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000,2412.000000,2412.000000,2448.000000,2399.000000,2448.000000,2448.000000,2448.000000,2399.000000,2448.000000,2.499000e+03,2499.000000,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2448.000000,2499.000000,2142.000000
mean,2019.591837,3.448980,2824.558007,7060.241830,2284.300245,1459.725899,3135.892565,3325.332108,4070.979575,6639.875622,4456.941957,5076.030637,1772.805336,6226.019199,2942.428513,12168.808415,1285.312630,7092.777778,1.268528e+06,177804.455508,6.889513e+05,2.696526e+05,1.321452e+05,8.002266e+05,4.684120e+05,1.951743e+05,3.617878e+05,3.104145e+05,4.013692e+05,4.918784e+05,7.767242e+05,4.181640e+05,4.379868e+05,4.123839e+05,0.055556,0.039216,0.274510
std,2.355825,1.727219,3222.248529,7991.012564,3057.111689,1847.997979,3720.934130,3656.871243,4598.127448,6896.744852,4879.998356,6280.243770,2040.574111,5186.780884,6351.094559,14034.375611,2254.539681,7895.294888,1.471416e+06,214064.564896,5.788009e+05,6.151876e+05,2.320144e+05,8.630926e+05,6.388622e+05,2.412678e+05,4.133378e+05,3.574091e+05,4.737732e+05,5.612155e+05,9.138216e+05,4.755620e+05,5.174024e+05,4.794179e+05,0.229108,0.194147,0.446371
min,2016.000000,1.000000,98.000000,438.000000,123.000000,46.000000,160.000000,163.000000,254.000000,90.000000,58.000000,253.000000,10.000000,383.000000,10.000000,745.000000,26.000000,442.000000,1.075990e+05,1057.000000,5.110500e+04,3.001000e+03,5.911000e+03,7.065300e+04,1.905287e+04,1.016233e+04,2.849000e+04,1.756665e+04,2.538500e+04,3.791700e+04,5.619400e+04,3.534700e+04,3.468144e+04,3.405700e+04,0.000000,0.000000,0.000000
25%,2018.000000,2.000000,833.000000,1682.000000,519.250000,400.500000,834.000000,993.250000,894.000000,1657.750000,1063.000000,1252.000000,135.000000,1876.000000,443.000000,3103.250000,332.500000,1710.500000,3.203580e+05,11895.000000,2.185310e+05,3.429900e+04,2.496900e+04,2.017420e+05,8.838300e+04,5.260900e+04,1.045050e+05,8.776700e+04,9.289700e+04,1.282931e+05,1.855815e+05,1.079980e+05,1.050790e+05,1.057575e+05,0.000000,0.000000,0.000000
50%,2020.000000,3.000000,2004.000000,4994.000000,1280.500000,1003.500000,2023.000000,2554.500000,2469.000000,4863.000000,3337.000000,3594.000000,795.000000,5056.500000,1131.000000,8834.500000,682.000000,4599.500000,8.527540e+05,78366.000000,5.483020e+05,8.945653e+04,6.349985e+04,5.384850e+05,3.385820e+05,1.364880e+05,2.646150e+05,2.176910e+05,2.407100e+05,3.489650e+05,5.042960e+05,2.882530e+05,2.907730e+05,2.848840e+05,0.000000,0.000000,0.000000
75%,2022.000000,5.000000,3495.500000,8323.250000,2911.000000,1675.750000,3818.500000,3957.000000,5339.500000,8415.000000,5608.250000,5799.750000,2919.500000,8585.000000,2458.250000,14239.000000,1541.500000,8866.750000,1.483094e+06,295449.500000,9.348310e+05,2.167530e+05,1.567080e+05,1.112506e+06,5.074350e+05,2.274260e+05,4.315310e+05,3.543330e+05,5.506045e+05,6.468486e+05,9.134280e+05,4.746365e+05,5.493285e+05,4.863455e+05,0.000000,0.000000,1.000000
max,2024.000000,6.000000,21744.000000,49042.000000,19493.000000,13024.000000,21410.000000,23035.000000,25989.000000,53083.000000,33183.000000,37677.000000,8816.000000,23550.000000,41101.000000,86129.000000,18225.000000,48561.000000,8.069261e+06,885854.973593,2.537353e+06,3.604176e+06,1.622266e+06,4.229409e+06,4.368042e+06,1.469984e+06,2.221492e+06,2.118518e+06,2.765170e+06,3.1

In [190]:
df2.describe()

,year,bmcode,births_age1524,births_age2534,births_age3544,births_nohs,births_hs,births_somecoll,births_coll,births_married,births_unmarried,births_medicaid,births_nhblack,births_nhwhite,births_hisp,births_total,births_otherraceeth,births_nonmedicaid,pop_total,pop_nhblack,pop_nhwhite,pop_hisp,pop_otherraceeth,pop_medicaid,pop_nonmedicaid,pop_nohs,pop_hs,pop_somecoll,pop_coll,pop_married,pop_unmarried,pop_age1524,pop_age2534,pop_age3544,policy_on,exposed_births,dobbs_code
count,2499.000000,2499.000000,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000,2448.000000,2412.000000,2412.000000,2448.000000,2408.000000,2448.000000,2448.000000,2448.000000,2408.000000,2448.000000,2.499000e+03,2499.000000,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2.499000e+03,2448.000000,2499.000000,2142.000000
mean,2019.591837,3.448980,2819.892157,7065.420752,2292.791258,1461.610703,3132.676879,3315.716912,4082.851307,6652.247098,4453.460614,5062.538399,1758.098837,6222.691176,2957.290850,12178.104167,1286.948920,7120.409314,1.268528e+06,177804.455508,6.889513e+05,2.696526e+05,1.321452e+05,8.002266e+05,4.684120e+05,1.951743e+05,3.617878e+05,3.104145e+05,4.013692e+05,4.918784e+05,7.767242e+05,4.181640e+05,4.379868e+05,4.123839e+05,0.055556,0.039216,0.274510
std,2.355825,1.727219,3217.999865,7995.446967,3068.880708,1854.004954,3707.017738,3649.426581,4615.877955,6911.626110,4874.996566,6260.203180,2030.490337,5184.312718,6362.594033,14044.338137,2260.449267,7931.368043,1.471416e+06,214064.564897,5.788009e+05,6.151876e+05,2.320144e+05,8.630926e+05,6.388622e+05,2.412678e+05,4.133378e+05,3.574091e+05,4.737732e+05,5.612155e+05,9.138216e+05,4.755620e+05,5.174024e+05,4.794179e+05,0.229108,0.194147,0.446371
min,2016.000000,1.000000,103.000000,434.000000,123.000000,46.000000,160.000000,138.000000,258.000000,90.000000,58.000000,253.000000,0.000000,372.000000,10.000000,755.000000,26.000000,442.000000,1.075990e+05,1057.000000,5.110500e+04,3.001000e+03,5.911000e+03,7.065300e+04,1.905287e+04,1.016233e+04,2.849000e+04,1.756665e+04,2.538500e+04,3.791700e+04,5.619400e+04,3.534700e+04,3.468144e+04,3.405700e+04,0.000000,0.000000,0.000000
25%,2018.000000,2.000000,830.750000,1682.000000,515.000000,389.750000,831.500000,984.000000,900.750000,1659.500000,1070.500000,1244.750000,131.000000,1875.250000,444.750000,3095.750000,328.000000,1715.750000,3.203580e+05,11895.000000,2.185310e+05,3.429900e+04,2.496900e+04,2.017420e+05,8.838300e+04,5.260900e+04,1.045050e+05,8.776700e+04,9.289700e+04,1.282931e+05,1.855815e+05,1.079980e+05,1.050790e+05,1.057575e+05,0.000000,0.000000,0.000000
50%,2020.000000,3.000000,2004.000000,4995.000000,1284.000000,1009.500000,2015.000000,2550.500000,2472.500000,4863.000000,3346.000000,3595.500000,789.000000,5056.500000,1152.500000,8812.500000,679.500000,4611.500000,8.527540e+05,78366.000000,5.483020e+05,8.945653e+04,6.349985e+04,5.384850e+05,3.385820e+05,1.364880e+05,2.646150e+05,2.176910e+05,2.407100e+05,3.489650e+05,5.042960e+05,2.882530e+05,2.907730e+05,2.848840e+05,0.000000,0.000000,0.000000
75%,2022.000000,5.000000,3495.500000,8334.750000,2918.000000,1675.750000,3811.250000,3951.000000,5341.250000,8436.000000,5607.250000,5765.250000,2885.000000,8585.000000,2482.750000,14242.250000,1545.000000,8863.500000,1.483094e+06,295449.500000,9.348310e+05,2.167530e+05,1.567080e+05,1.112506e+06,5.074350e+05,2.274260e+05,4.315310e+05,3.543330e+05,5.506045e+05,6.468486e+05,9.134280e+05,4.746365e+05,5.493285e+05,4.863455e+05,0.000000,0.000000,1.000000
max,2024.000000,6.000000,21744.000000,49042.000000,19493.000000,13024.000000,21238.000000,23035.000000,25989.000000,53083.000000,33183.000000,37677.000000,8816.000000,23550.000000,41101.000000,86129.000000,18225.000000,48561.000000,8.069261e+06,885854.973593,2.537353e+06,3.604176e+06,1.622266e+06,4.229409e+06,4.368042e+06,1.469984e+06,2.221492e+06,2.118518e+06,2.765170e+06,3.1